In [1]:
import time
import pickle
import math
import numpy as np 
import torch
from torch import nn
from torch.autograd import Variable
import RSPlanner
import data_loader
import model
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
def in_rectangle(x, y, rect, ratio = 1/256):
    x_p = (ratio*x - rect[0]) * np.cos(-rect[2]) - (ratio*y - rect[1]) * np.sin(-rect[2])
    y_p = (ratio*x - rect[0]) * np.sin(-rect[2]) + (ratio*y - rect[1]) * np.cos(-rect[2])
    if abs(x_p) <= ((1/10)/2) and abs(y_p) <= ((1/20)/2):
        return True
    else:
        return False
    

def IsInCollision(x, obs):
    
        for a, b in zip(np.where(obs == 1)):
            if in_rectangle(b, 255-a, x):
                return True # in collision
                
        return False  # safe

In [3]:
def steerTo (start, end, obs):

    path = RSPlanner.main(start, end, obs)

    if path is None:
        return 0
    else:
        return 1

In [4]:
def feasibility_check(path, obs):

    for i in range(0, len(path)-1):
        ind = steerTo(path[i], path[i+1], obs)
        if ind == 0:
            return 0
    return 1

In [5]:
def collision_check(path, obs):

    for i in range(0, len(path)):
        if IsInCollision(path[i], obs):
            return 0
    return 1

In [6]:
def is_reaching_target(start1, start2):
    
    s1=np.zeros(2,dtype=np.float32)
    s1[0]=start1[0]
    s1[1]=start1[1]

    s2=np.zeros(2,dtype=np.float32)
    s2[0]=start2[0]
    s2[1]=start2[1]

    for i in range(0,2):
        if abs(s1[i]-s2[i]) > 0.01: 
            return False
    return True

In [7]:
#lazy vertex contraction 
def lvc(path, obs):

    for i in range(0,len(path) - 1):
        for j in range(len(path)-1, i+1, -1):
            ind=0
            ind=steerTo(path[i], path[j], obs)
            if ind==1:
                pc=[]
                for k in range(0,i+1):
                    pc.append(path[k])
                for k in range(j,len(path)):
                    pc.append(path[k])

                return lvc(pc, obs)

    return path

In [8]:
def re_iterate_path2(p, g, obs, z, pnet):
    step=0
    path=[]
    path.append(p[0])
    for i in range(1,len(p)-1):
        if not IsInCollision(p[i], obs):
            path.append(p[i])
    path.append(g)
    new_path=[]
    for i in range(0,len(path)-1):
        target_reached=False

        st=path[i]
        gl=path[i+1]
        steer=steerTo(st, gl, obs)
        if steer==1:
            new_path.append(st)
            new_path.append(gl)
        else:
            itr=0
            target_reached=False
            while (not target_reached) and itr<50 :
                new_path.append(st)
                itr=itr+1
                ip=torch.cat((z, st.cuda(), gl.cuda()), 1)
                ip=Variable(ip, volatile = False)
                st=pnet(ip).data.cpu()
                target_reached=is_reaching_target(st, gl)
            if target_reached==False:
                return 0

    #new_path.append(g)
    return new_path


In [9]:
def replan_path(p, g, obs, z, pnet):
    step=0
    path=[]
    path.append(p[0])
    for i in range(1,len(p)-1):
        if not IsInCollision(p[i], obs):
            path.append(p[i])
    path.append(g)
    new_path=[]
    for i in range(0,len(path)-1):
        target_reached=False

        st=path[i]
        gl=path[i+1]
        steer=steerTo(st, gl, obs)
        if steer==1:
            new_path.append(st)
            new_path.append(gl)
        else:
            itr=0
            pA=[]
            pA.append(st)
            pB=[]
            pB.append(gl)
            target_reached=0
            tree=0
            while target_reached==0 and itr<50 :
                itr=itr+1
                if tree==0:
                    ip1=torch.cat((z, st ,gl), 1)
                    ip1=Variable(ip1, volatile = False)
                    st=pnet(ip1).data.cpu()
                    pA.append(st)
                    tree=1
                else:
                    ip2=torch.cat((z, gl, st), 1)
                    ip2=Variable(ip2, volatile = False)
                    gl=pnet(ip2).data.cpu()
                    pB.append(gl)
                    tree=0
                target_reached=steerTo(st, gl, obs)
            if target_reached==0:
                return 0
            else:
                for p1 in range(0,len(pA)):
                    new_path.append(pA[p1])
                for p2 in range(len(pB)-1,-1,-1):
                    new_path.append(pB[p2])

    return new_path

In [10]:
def plan(start, goal, obs, model):
    
    enet = model[0]
    pnet = model[1]
    
    if torch.cuda.is_available():
        obs = obs.cuda()
        enet = enet.cuda()
        pnet = pnet.cuda()
    enet.train()
    pnet.train()
        
    #encode occupancy map
    z = enet(obs)
    
    ## starting point
    start1 = start.cuda()
    goal2 = start.cuda()
    ##goal point
    goal1 = goal.cuda()
    start2 = goal.cuda()
    ##generated paths
    path1 = [] 
    path1.append(start1)
    path2 = []
    path2.append(start2)
    target_reached = 0
    step = 0
    path = [] # stores end2end path by concatenating path1 and path2
    tree = 0
    
    while target_reached == 0 and step < 80 :
        step = step + 1
        if tree == 0:
            inp1 = torch.cat((z, start1, start2), 1)
            inp1 = Variable(inp1, volatile = False)
            start1 = pnet(inp1).data.cpu()
            path1.append(start1)
            tree = 1
        else:
            inp2 = torch.cat((z, start2,start1), 1)
            inp2 = Variable(inp2, volatile = False)
            start2 = pnet(inp2).data.cpu()
            path2.append(start2)
            tree = 0
        target_reached = steerTo(start1,start2,i);
    
    if target_reached == 1:
        for p1 in range(0,len(path1)):
            path.append(path1[p1])
        for p2 in range(len(path2)-1,-1,-1):
            path.append(path2[p2])
            
        path=lvc(path, obs.cpu())
        indicator=feasibility_check(path, obs.cpu())
        if indicator==1:
            toc = time.clock()
            t=toc-tic
            et.append(t)
            fp=fp+1
            print ("path[0]:")
            for p in range(0,len(path)):
                print (path[p][0])
            print ("path[1]:")
            for p in range(0,len(path)):
                print (path[p][1])
#             print ("Actual path[0]:")
#             for p in range(0,path_lengths[i][j]):
#                 print (paths[i][j][p][0])
#             print ("Actual path[1]:")
#             for p in range(0,path_lengths[i][j]):
#                 print (paths[i][j][p][1])
        else:
            sp=0
            indicator=0
            while indicator==0 and sp<10 and path !=0:
                sp=sp+1
                g=np.zeros(2,dtype=np.float32)
                g=torch.from_numpy(goal)
                path=replan_path(path, g, obs.cpu(), z, pnet) #replanning at coarse level
                if path !=0:
                    path=lvc(path, obs.cpu())
                    indicator=feasibility_check(path, obs.cpu())
                if indicator==1:
                    toc = time.clock()
                    t=toc-tic
                    et.append(t)
                    fp=fp+1
                    if len(path)<20:
                        print ("new_path[0]:")
                        for p in range(0,len(path)):
                            print (path[p][0])
                        print ("new_path[1]:")
                        for p in range(0,len(path)):
                            print (path[p][1])
#                         print ("Actual path[0]:")
#                         for p in range(0,path_lengths[i][j]):
#                             print (paths[i][j][p][0])
#                         print ("Actual path[1]:")
#                         for p in range(0,path_lengths[i][j]):
#                             print (paths[i][j][p][1])
                    else:
                        print ("path found, dont worry")
            
    